In [203]:
import os, re
from string import punctuation
import numpy as np
import pandas as pd
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import textdistance
import numpy as np
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [271]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

In [267]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=4, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
    # мой код
    closest_words_with_probs = {word:[distance, P(word)] for word, distance in dict(closest).items()}
    cl_df = pd.DataFrame.from_dict(closest_words_with_probs, orient='index', columns=['dist', 'prob'])
    cl_df = cl_df.sort_values(by=['dist', 'prob'], ascending=[False, False]).drop_duplicates(['dist'])

    return closest, cl_df.to_dict(orient='index')
    

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

In [268]:
without_prob_filtering_words, with_prob_filtering_words = get_closest_hybrid_match('сопнце', X, vec)

In [277]:
without_prob_filtering_words

[('солнце', 0.8333333333333334),
 ('солнцем', 0.7142857142857143),
 ('солнцев', 0.7142857142857143),
 ('сопернице', 0.6666666666666667)]

In [278]:
with_prob_filtering_words

{'солнце': {'dist': 0.8333333333333334, 'prob': 2.4440966240624417e-05},
 'солнцем': {'dist': 0.7142857142857143, 'prob': 8.340964669419444e-06},
 'сопернице': {'dist': 0.6666666666666667, 'prob': 3.879518450892764e-07}}

In [274]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], list(get_closest_hybrid_match(pair[1], X, vec)[1].items())[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1
    
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [276]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.856328164082041
0.4937888198757764
0.09004249454461927


Работает улучшенный алгоритм действительно лучше, на 3% ошибок исправлено больше, чем в алгоритме, который был первоначально.

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [1]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [4]:
# создаем словарь
vocab = Counter(re.findall('\w+', corpus.lower()))

In [20]:
WORDS = list(vocab)

In [5]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

In [29]:
# оригинальный код вот тут - 
# я только адаптировал его под русский язык

def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [37]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [38]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [39]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8479239619809905
0.15527950310559005
0.0496152520960147


Процент исправленных ошибок снизился в три раза по сравнению с тем вариантом, когда мы используем все варианты замен, вставок, перестановок и т.д. Скорость исполнения алгоритма очень низкая.

## *3. Настройка гиперпараметров. (2 балла)

У метода из первого заданий много гиперпараметров (те которые нужно подбирать самостоятельно). Это параметры векторайзера, topn, метрика редактирования. Поэкспериментируйте с ними. 

Проведите как минимум 10 экспериментов с разными параметрами. Для каждого эксперимента укажите мотивацию (например, "слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить")

Старайтесь получить улучшение, но если улучшить не получится, то это не страшно. Главное, чтобы эксперименты были осмысленными, а не рандомными. 

In [214]:
%%time
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

without_prob_filtering_words, with_prob_filtering_words = get_closest_hybrid_match('сонцее', X, vec)
with_prob_filtering_words

Wall time: 8.43 s


{'солнцем': {'dist': 0.7142857142857143, 'prob': 8.340964669419444e-06},
 'солнце': {'dist': 0.6666666666666667, 'prob': 2.4440966240624417e-05},
 'солнцецвет': {'dist': 0.6, 'prob': 1.939759225446382e-07},
 'саксонцев': {'dist': 0.5555555555555556, 'prob': 5.819277676339147e-07},
 'ньонце': {'dist': 0.5, 'prob': 1.939759225446382e-07},
 'бессоннице': {'dist': 0.4, 'prob': 3.879518450892764e-07},
 'шнеерсон': {'dist': 0.125, 'prob': 3.879518450892764e-07}}

In [215]:
%%time
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=2000)
X = vec.fit_transform(vocab)

without_prob_filtering_words, with_prob_filtering_words = get_closest_hybrid_match('сонцее', X, vec)
with_prob_filtering_words

Wall time: 8.28 s


{'монцезе': {'dist': 0.7142857142857143, 'prob': 1.939759225446382e-07},
 'конце': {'dist': 0.6666666666666667, 'prob': 0.00037068798798280367},
 'саксонцев': {'dist': 0.5555555555555556, 'prob': 5.819277676339147e-07},
 'ньонце': {'dist': 0.5, 'prob': 1.939759225446382e-07},
 'шнеерсон': {'dist': 0.125, 'prob': 3.879518450892764e-07}}